# Simple example demonstrating use of tred library
This is a rough demonstration of the basic functionalities
in `tred`. It unfortunately, at the moment, does not cover all of the public 
functionality. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from numpy.testing import assert_allclose

import tred
from tred import display_tensor_facewise as disp

First we will create some dummy data

In [3]:
# let's use a numpy rng
RNG = np.random.default_rng(seed=1234)

# then we will generate some dummy data
n, p, t = 3, 4, 2
X = RNG.random(size=(n, p, t))

Let's quickly view our data using the `display_tensor_facewise` function, which we imported as `disp`. Note: this looks a bit nicer than calling `print(X)`

In [4]:
disp(X)

Tensor with dimensions (3, 4, 2)
[[[0.97669977 0.92324623 0.31909706 0.24176629]
  [0.96407925 0.44100612 0.8636213  0.67488131]
  [0.7357577  0.17206618 0.06013866 0.67123802]]

 [[0.38019574 0.26169242 0.11809123 0.31853393]
  [0.2636498  0.60987081 0.86375767 0.65987435]
  [0.22275366 0.87041497 0.68368891 0.61101798]]]


## tsvdm tensor decomposition algorithm
Let's get the tsvdm decomposition first, using the `tsvdm` function

In [5]:
# so, first we can use tred's tsvdm decomposition
# NOTE: we return V not Vt
U, S, V = tred.tsvdm(X)

In [6]:
disp(U)

Tensor with dimensions (3, 3, 2)
[[[ 0.17155477 -0.03254445 -0.56750142]
  [ 0.77376295 -0.77505461  0.24995519]
  [ 1.00613179  0.03661673  0.27647539]]

 [[ 0.49697207  1.1805333   0.08253392]
  [ 0.21963571 -0.04814225 -0.82918402]
  [-0.25364858  0.03023034  0.91905817]]]


In [7]:
disp(S)

Tensor with dimensions (3, 4, 2)
[[[ 2.49071429  0.          0.          0.        ]
  [ 0.          0.95133804  0.          0.        ]
  [ 0.          0.          0.33545775  0.        ]]

 [[ 1.2959226   0.          0.          0.        ]
  [ 0.         -0.06058465  0.          0.        ]
  [ 0.          0.          0.10984777  0.        ]]]


In [8]:
disp(V)

Tensor with dimensions (4, 4, 2)
[[[ 0.72770621 -0.26446627 -0.16542903 -0.41692012]
  [-0.15139073 -0.0209886  -0.29168408  0.60752868]
  [-0.01705587 -0.61095445  0.23509181  0.09922585]
  [ 0.39762985 -0.16678187  0.60880143  0.52113841]]

 [[ 0.0244148   0.95052807 -0.33592066 -0.4270477 ]
  [ 0.84344351  0.65517601  0.43308422  0.44074385]
  [ 0.69522119 -0.36615433 -0.96110795  0.14212615]
  [ 0.30626555 -0.2484096   0.48735324 -0.8825139 ]]]


In [9]:
# NOTE: the V tensor is not transposed (facewise)!
# in order to do a facewise transpose - use numpy
Vt = V.transpose(1, 0, 2)

In [10]:
disp(Vt)

Tensor with dimensions (4, 4, 2)
[[[ 0.72770621 -0.15139073 -0.01705587  0.39762985]
  [-0.26446627 -0.0209886  -0.61095445 -0.16678187]
  [-0.16542903 -0.29168408  0.23509181  0.60880143]
  [-0.41692012  0.60752868  0.09922585  0.52113841]]

 [[ 0.0244148   0.84344351  0.69522119  0.30626555]
  [ 0.95052807  0.65517601 -0.36615433 -0.2484096 ]
  [-0.33592066  0.43308422 -0.96110795  0.48735324]
  [-0.4270477   0.44074385  0.14212615 -0.8825139 ]]]


Let's also verify that we can use the tsvdm decomposition to reconstruct the 
original tensor `X`.

In [11]:
# see documentation - allows us to apply a function over a sequence of inputs 
# cumulatively from left to right
from functools import reduce

# tred has a function to comput the m_product, but it does not make default assumptions
# about the m-transform - as the user, you need to write a simple wrapper to make it work
# with a m-product you want

# you can fetch the default m-transforms used by tsvdm and TPCA using the following tred 
# function
M, Minv = tred.generate_default_m_transform_pair(t)

# then write our wrapper function
def m_product_wrapper(A, B): 
    return tred.m_product(A, B, M=M, Minv=Minv)

# multiply U, S, Vt together using the m-product and default m-transfrom
X_reconstructed = reduce(m_product_wrapper, (U, S, Vt))
assert_allclose(X_reconstructed, X)

Notice how the last column of `S` is all zeroes. This means that some of the
`V` tensor is redundant. We can pass in `full_frontal_slices=False` to only
return us the necessary elements in the tensor.

In [12]:
U2, S2, V2 = tred.tsvdm(X, full_frontal_slices=False)

(U2.shape, S2.shape, V2.shape)

((3, 3, 2), (3, 3, 2), (4, 3, 2))

Sense check that we have not 'loss' any information, and we can still 
fully reconstruct our original tensor

In [13]:
V2t = V2.transpose(1, 0, 2)
X_reconstructed2 = reduce(m_product_wrapper, (U2, S2, V2t))
assert_allclose(X_reconstructed2, X)

## TPCA unsupervised tensor dimensionality reduction algorithm
Now, let's use the tensor decomposition using the `tpca` object. Revise some basics of object oriented programming and scikit-learn's interface if you are coming from R.

In [14]:
# first, create a transformer object
tpca = tred.TPCA()

There's two ways to do this:

1. You explicitly fit your tpca object _first_, and then transform your data.
Let's see what happens if you call `transform` _before_ `fit`.

In [15]:
# ERROR
X_transformed_1 = tpca.transform(X)

NotFittedError: This TPCA instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# so, we need to fit the tpca object first - then call transform!
tpca.fit(X)
X_transformed_1 = tpca.transform(X)

In [ ]:
# the size of this is n rows by min(n, p) columns
X_transformed_1

array([[ 5.68460112e-01,  5.11808598e-01, -1.54348130e-01,
        -6.89881787e-02,  1.42247325e-16, -2.77555756e-17],
       [-6.66509430e-03, -5.59723785e-01, -1.17834904e-01,
         1.40461250e-01, -1.70870262e-16, -2.77555756e-17],
       [-5.61795018e-01,  4.79151878e-02,  2.72183034e-01,
        -7.14730711e-02,  1.73472348e-17,  1.80411242e-16]])

2. Use `fit_transform`

NOTE: this produces the same result to machine precision, but it's not identical, as we 
use a slightly more efficient computation to obtain the transformed data. 

See the `tred` library implementation for more details

In [ ]:
X_transformed_2 = tpca.fit_transform(X)

In [ ]:
# NOTE: this is the same as the array above - to machine precision
# some numbers LOOK different, but you'll see that they are tiny and
# are effectively the same
X_transformed_2

array([[ 5.68460112e-01,  5.11808598e-01, -1.54348130e-01,
        -6.89881787e-02, -6.00850325e-17,  5.86791256e-17],
       [-6.66509430e-03, -5.59723785e-01, -1.17834904e-01,
         1.40461250e-01, -6.00850325e-17,  5.86791256e-17],
       [-5.61795018e-01,  4.79151878e-02,  2.72183034e-01,
        -7.14730711e-02, -6.00850325e-17,  5.86791256e-17]])

In [ ]:
# the tensor operations produce some floating point errors
# below are the tolerances we use in tred's testing module, which show that the two
# transformations of X are indeed the same
RTOL = 1e-7
ATOL = 1e-10
assert_allclose(X_transformed_1, X_transformed_2, rtol=RTOL, atol=ATOL)

We can use the fitted objects to transform out of bag data

In [ ]:
# imagine one new observation, with p features, across t time points
new_obs = RNG.random(size=(1, p, t))

In [ ]:
disp(new_obs)

Tensor with dimensions (1, 4, 2)
[[[0.06013731 0.43895163 0.00313229 0.85849044]]

 [[0.97776927 0.53259502 0.25126711 0.42529835]]]


In [ ]:
tpca.transform(new_obs)

array([[-0.13546024,  0.34661962,  0.39718925, -0.55304217,  0.95086191,
         0.07885213]])

Make sure you do not call `fit` or `fit_transform` on the new data, otherwise 
the `tpca` object will learn new weights!